In [ ]:
!pip install openpyxl
!pip install pandas

In [11]:
import pandas as pd
import sqlite3
import openpyxl
import json
import sys
sys.path.append("..")
from utils.github_utils import GithubUtils
from utils.java_code_cleaner import JavaCodeCleaner
from utils.code_smells import CodeSmells
from utils.dataset_utils import *

In [15]:
tech_debt_db_path = "./input/tech_debt.db"

conn = sqlite3.connect(tech_debt_db_path)

cursor = conn.cursor()

sonar_issues_query = """
    SELECT 
        SONAR_ISSUES.PROJECT_ID, 
        GIT_LINK,
        REVISION,
        COMPONENT,
        MESSAGE,
        SEVERITY
    --     STATUS,
    --     RESOLUTION,
    --     CREATION_ANALYSIS_KEY
    --     CLOSE_ANALYSIS_KEY,
    --     ISSUE_KEY,
    --     CREATION_DATE,
    --     CLOSE_DATE
    FROM 
        SONAR_ISSUES
            JOIN PROJECTS
                ON SONAR_ISSUES.PROJECT_ID = PROJECTS.PROJECT_ID
                    JOIN SONAR_ANALYSIS 
                        ON SONAR_ISSUES.CREATION_ANALYSIS_KEY = SONAR_ANALYSIS.ANALYSIS_KEY
    WHERE COMPONENT LIKE '%.JAVA' 
        AND TYPE = 'CODE_SMELL' 
        AND RULE LIKE 'code_smell%';
"""

# Execute a query (example: select all rows from a table)
cursor.execute(sonar_issues_query)

# Fetch all results
rows = cursor.fetchall()

# Loop through the results and print them
# for row in rows:
#     print(row)

cursor.close()
conn.close()

In [16]:
i = 0
while i < 10:
    row = rows[i]
    print(row)
    # print(row[0])
    i = i + 1

('org.apache:cayenne', 'https://github.com/apache/cayenne', 'fc186f50392bf8c804b1d314971c92efa5f52fc3', 'Apache_Cayenne:framework/cayenne-jdk1.4-unpublished/src/main/java/org/apache/cayenne/xml/XMLEncoder.java', 'Complex class', 'MINOR')
('org.apache:cayenne', 'https://github.com/apache/cayenne', 'fc186f50392bf8c804b1d314971c92efa5f52fc3', 'Apache_Cayenne:framework/cayenne-jdk1.4-unpublished/src/main/java/org/apache/cayenne/access/DataContextFaults.java', 'Complex class', 'MINOR')
('org.apache:cayenne', 'https://github.com/apache/cayenne', 'fc186f50392bf8c804b1d314971c92efa5f52fc3', 'Apache_Cayenne:framework/cayenne-jdk1.4-unpublished/src/main/java/org/apache/cayenne/util/XMLEncoder.java', 'Complex class', 'MINOR')
('org.apache:cayenne', 'https://github.com/apache/cayenne', 'fc186f50392bf8c804b1d314971c92efa5f52fc3', 'Apache_Cayenne:framework/cayenne-jdk1.4-unpublished/src/main/java/org/apache/cayenne/exp/parser/TokenMgrError.java', 'Complex class', 'MINOR')
('org.apache:cayenne', 'htt

In [18]:
#! Cols names:
id_col = "id"
github_repo_url_col = "github_repo_url"
hash_commit_col = "hash_commit"
file_name_col = "file_name"
code_smell_col = "code_smell"
severity_col = "severity"

intermediate_tech_debt_dataset = {
    id_col: [],  # A sequential Id
    github_repo_url_col: [],  # Github normal URL of the instance
    hash_commit_col: [],  # Github Raw URL of the instance
    file_name_col: [],  # 1 If Instance has a Blob Smell, Otherwise 0
    code_smell_col: [],  # 1 If Instance has a DataClass Smell, Otherwise 0
    severity_col: []  # 1 If Instance has a DataClass Smell, Otherwise 0
}

i = 1

for row in rows:

    intermediate_tech_debt_dataset[id_col].append(i)
    intermediate_tech_debt_dataset[github_repo_url_col].append(row[1])
    intermediate_tech_debt_dataset[hash_commit_col].append(row[2])
    intermediate_tech_debt_dataset[file_name_col].append(row[3])
    intermediate_tech_debt_dataset[code_smell_col].append(row[4])
    intermediate_tech_debt_dataset[severity_col].append(row[5])
    
    i = i + 1


intermediate_tech_debt_dataset_df = pd.DataFrame(intermediate_tech_debt_dataset)


#! Save Intermediate Dataset to a CSV file
intermediate_tech_debt_dataset_csv_path = "./output/v1/tech_debt_intermediate_dataset.csv"
intermediate_tech_debt_dataset_df.to_csv(intermediate_tech_debt_dataset_csv_path, index=False)


#! Save Intermediate Dataset to a XLSX file
intermediate_tech_debt_dataset_xlsx_path = "./output/v1/tech_debt_intermediate_dataset.xlsx"
intermediate_tech_debt_dataset_df.to_excel(
    intermediate_tech_debt_dataset_xlsx_path, index=False, engine='openpyxl')

In [ ]:
#! Cols names:
id_col = "id"
github_url_col = "github_url"
github_raw_url_col = "github_raw_url"
code_smell_col = "code_smell"
severity_col = "severity"

intermediate_dataset = {
    id_col: [],  # A sequential Id
    github_url_col: [],  # Github normal URL of the instance
    github_raw_url_col: [],  # Github Raw URL of the instance
    code_smell_col: [],  # Code Smell Name
    severity_col: []  # Severity Minor, Major, Critical
}

intermediate_tech_debt_dataset_csv_path = "./output/v1/tech_debt_intermediate_dataset.csv"

intermediate_tech_debt_dataset_df = pd.read_csv(
    intermediate_tech_debt_dataset_csv_path, on_bad_lines="skip")


for i, row in intermediate_tech_debt_dataset_df.iterrows():
    intermediate_dataset[id_col].append(i // 2 + 1)
    intermediate_dataset[github_url_col].append(
        row["github_repo_url"] + "/blob/" + row["hash_commit"] + row["file_name"].split(":")[-1])
    intermediate_dataset[github_raw_url_col].append(
        GithubUtils.convert_blob_to_raw_url(row["github_repo_url"] + "/blob/" + row["hash_commit"] + "/" + row["file_name"].split(":")[1]))
    intermediate_dataset[code_smell_col].append(row["code_smell"])
    intermediate_dataset[severity_col].append(row["severity"])
    

intermediate_dataset_df = pd.DataFrame(intermediate_dataset)


#! Save Intermediate Dataset to a CSV file
intermediate_tech_debt_dataset_csv_path = "./output/v2/tech_debt_intermediate_dataset.csv"
intermediate_dataset_df.to_csv(
    intermediate_tech_debt_dataset_csv_path, index=False)


#! Save Intermediate Dataset to a XLSX file
intermediate_tech_debt_dataset_xlsx_path = "./output/v2/tech_debt_intermediate_dataset.xlsx"
intermediate_dataset_df.to_excel(
    intermediate_tech_debt_dataset_xlsx_path, index=False, engine='openpyxl')